### Display Countries

In [1]:
import os
import pandas as pd
import numpy as np 
import math

from ast import literal_eval
from pathlib import Path

from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Viridis5
from bokeh.models import ColumnDataSource, ColorBar, BasicTicker, HoverTool
from bokeh.models.mappers import ColorMapper, LinearColorMapper

In [2]:
df = pd.read_csv("data/countries.csv", index_col='id')
df = df.dropna()
df.head()

,alpha_2_code,latitude,longitude,country,alpha_3_code
id,,,,,
0,AD,42.546245,1.601554,Andorra,AND
1,AE,23.424076,53.847818,United Arab Emirates,ARE
2,AF,33.939110,67.709953,Afghanistan,AFG
3,AG,17.060816,-61.796428,Antigua and Barbuda,ATG
4,AI,18.220554,-63.068615,Anguilla,AIA


In [3]:
df.loc[df['country'] == 'Angola']

,alpha_2_code,latitude,longitude,country,alpha_3_code
id,,,,,
7,AO,-11.202692,17.873887,Angola,AGO


In [4]:
# Convert latitude and longitude into mercator projection mapping
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

def make_tuple_str(x, y):
    t = (x, y)
    return str(t)

# Useful link: https://getlatlong.net
range0 = merc('(12.080577, -64.052642)')
range1 = merc('(82.356802, -184.753326)')
x_range = (range0[0],range1[0])
y_range = (range0[1], range1[1])

# Convert DataFrame longitude and latitude column into mercator coordinates
df['coords'] = df.apply(lambda x: make_tuple_str(x['latitude'], x['longitude']), axis = 1)
df['coords_latitude'] = df['coords'].apply(lambda x: merc(x)[0])
df['coords_longitude'] = df['coords'].apply(lambda x: merc(x)[1])

# Prepare data
source = ColumnDataSource(
    data = dict(
            lat = df['coords_latitude'].tolist(),
            lon = df['coords_longitude'].tolist(),
            #size = 10,
            #color = 'blue',
            name = df['country'].tolist(),
        )
    )

select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']

# Format the tooltip
tooltips = [
            ('Name','@name'),
           ]

p = figure(x_range=x_range, y_range=y_range, x_axis_type="mercator", y_axis_type="mercator",
              plot_width=900, plot_height=700)
p.add_tile(get_provider(Vendors.CARTODBPOSITRON))

# Configure a renderer to be used upon hover
hover_glyph = p.circle(x='lat', y='lon', source=source,
                         size=10, alpha=0,
                         hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
p.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

# Color palette
color_mapper = LinearColorMapper(palette = Viridis5)

# Add a glyph
p.circle(x='lat', y='lon', 
        size=5,
        color='red',
        source=source 
        )

# Put a colorbar to support
color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

# Layout
p.add_layout(color_bar, 'right')

output_notebook()

show(p)


Loading BokehJS ...